**IMPORTING LIBRARIES**

In [1]:
import pandas as pd
import sklearn
import itertools
import numpy as np
import seaborn as sb
import re
import nltk
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from sklearn.linear_model import PassiveAggressiveClassifier
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

**DATA PREPERATION**

In [5]:
#data collection
#loading dataset into pandas dataframe
train_df = pd.read_csv(r'C:/Users/evilv/Downloads/Fake-News-Detection-using-MachineLearning-main/Fake-News-Detection-using-MachineLearning-main/dataset/train.csv')
train_df = pd.read_csv(r'C:/Users/evilv/Downloads/Fake-News-Detection-using-MachineLearning-main/Fake-News-Detection-using-MachineLearning-main/TamilFakeAndRealNew.')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/evilv/Downloads/Fake-News-Detection-using-MachineLearning-main/Fake-News-Detection-using-MachineLearning-main/TamilFakeAndRealNew.'

In [6]:
train_df.head(15)

id                                              title  \
0    0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1    1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2    2                  Why the Truth Might Get You Fired   
3    3  15 Civilians Killed In Single US Airstrike Hav...   
4    4  Iranian woman jailed for fictional unpublished...   
5    5  Jackie Mason: Hollywood Would Love Trump if He...   
6    6  Life: Life Of Luxury: Elton John’s 6 Favorite ...   
7    7  Benoît Hamon Wins French Socialist Party’s Pre...   
8    8  Excerpts From a Draft Script for Donald Trump’...   
9    9  A Back-Channel Plan for Ukraine and Russia, Co...   
10  10  Obama’s Organizing for Action Partners with So...   
11  11  BBC Comedy Sketch "Real Housewives of ISIS" Ca...   
12  12  Russian Researchers Discover Secret Nazi Milit...   
13  13  US Officials See No Link Between Trump and Russia   
14  14  Re: Yes, There Are Paid Government Trolls On S...   

                          author  \
0                  Darrell Lucus   
1                Daniel J. Flynn   
2             Consortiumnews.com   
3                Jessica Purkiss   
4                 Howard Portnoy   
5                Daniel Nussbaum   
6                            NaN   
7                Alissa J. Rubin   
8                            NaN   
9   Megan Twohey and Scott Shane   
10                   Aaron Klein   
11               Chris Tomlinson   
12                 Amando Flavio   
13                    Jason Ditz   
14                  AnotherAnnie   

                                                 text  label  
0   House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1   Ever get the feeling your life circles the rou...      0  
2   Why the Truth Might Get You Fired October 29, ...      1  
3   Videos 15 Civilians Killed In Single US Airstr...      1  
4   Print \nAn Iranian woman has been sentenced to...      1  
5   In these trying times, Jackie Mason is the Voi...      0  
6   Ever wonder how Britain’s most iconic pop pian...      1  
7   PARIS  —   France chose an idealistic, traditi...      0  
8   Donald J. Trump is scheduled to make a highly ...      0  
9   A week before Michael T. Flynn resigned as nat...      0  
10  Organizing for Action, the activist group that...      0  
11  The BBC produced spoof on the “Real Housewives...      0  
12  The mystery surrounding The Third Reich and Na...      1  
13  Clinton Campaign Demands FBI Affirm Trump's Ru...      1  
14  Yes, There Are Paid Government Trolls On Socia...      1

In [7]:
#deleting unwanted columns
train_df = train_df.drop("author", axis = 1)
train_df = train_df.drop("title", axis = 1)
train_df = train_df.drop("id", axis = 1)

In [8]:
train_df.shape

(20800, 2)

In [9]:
train_df.head(15)

text  label
0   House Dem Aide: We Didn’t Even See Comey’s Let...      1
1   Ever get the feeling your life circles the rou...      0
2   Why the Truth Might Get You Fired October 29, ...      1
3   Videos 15 Civilians Killed In Single US Airstr...      1
4   Print \nAn Iranian woman has been sentenced to...      1
5   In these trying times, Jackie Mason is the Voi...      0
6   Ever wonder how Britain’s most iconic pop pian...      1
7   PARIS  —   France chose an idealistic, traditi...      0
8   Donald J. Trump is scheduled to make a highly ...      0
9   A week before Michael T. Flynn resigned as nat...      0
10  Organizing for Action, the activist group that...      0
11  The BBC produced spoof on the “Real Housewives...      0
12  The mystery surrounding The Third Reich and Na...      1
13  Clinton Campaign Demands FBI Affirm Trump's Ru...      1
14  Yes, There Are Paid Government Trolls On Socia...      1

In [10]:
train_df.isna().sum()

text     39
label     0
dtype: int64

In [11]:
def create_distribution(dataFile):
    return sb.countplot(x='label', data=dataFile, palette='hls')

# by calling below we can see that training, test and valid data seems to be failry evenly distributed between the classes
create_distribution(train_df)

C:\Users\evilv\AppData\Local\Temp\ipykernel_4780\33152063.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  return sb.countplot(x='label', data=dataFile, palette='hls')


<Axes: xlabel='label', ylabel='count'>

In [12]:
def data_qualityCheck():
    print("Checking data qualitites...")
    train_df.isnull().sum()
    train_df.info()  
    print("check finished.")
data_qualityCheck()

Checking data qualitites...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20761 non-null  object
 1   label   20800 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 325.1+ KB
check finished.


In [13]:
train_df = train_df.dropna()

In [14]:
data_qualityCheck()

Checking data qualitites...
<class 'pandas.core.frame.DataFrame'>
Index: 20761 entries, 0 to 20799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20761 non-null  object
 1   label   20761 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 486.6+ KB
check finished.


In [15]:
train_df.shape

(20761, 2)

In [16]:
train_df.head(10)

text  label
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1
1  Ever get the feeling your life circles the rou...      0
2  Why the Truth Might Get You Fired October 29, ...      1
3  Videos 15 Civilians Killed In Single US Airstr...      1
4  Print \nAn Iranian woman has been sentenced to...      1
5  In these trying times, Jackie Mason is the Voi...      0
6  Ever wonder how Britain’s most iconic pop pian...      1
7  PARIS  —   France chose an idealistic, traditi...      0
8  Donald J. Trump is scheduled to make a highly ...      0
9  A week before Michael T. Flynn resigned as nat...      0

In [17]:
train_df.reset_index(drop= True,inplace=True)

In [18]:
train_df.head(10)

text  label
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1
1  Ever get the feeling your life circles the rou...      0
2  Why the Truth Might Get You Fired October 29, ...      1
3  Videos 15 Civilians Killed In Single US Airstr...      1
4  Print \nAn Iranian woman has been sentenced to...      1
5  In these trying times, Jackie Mason is the Voi...      0
6  Ever wonder how Britain’s most iconic pop pian...      1
7  PARIS  —   France chose an idealistic, traditi...      0
8  Donald J. Trump is scheduled to make a highly ...      0
9  A week before Michael T. Flynn resigned as nat...      0

In [19]:
label_train = train_df.label

In [20]:
label_train.head(10)

0    1
1    0
2    1
3    1
4    1
5    0
6    1
7    0
8    0
9    0
Name: label, dtype: int64

In [21]:
train_df = train_df.drop("label", axis = 1)

In [22]:
train_df.head(10)

text
0  House Dem Aide: We Didn’t Even See Comey’s Let...
1  Ever get the feeling your life circles the rou...
2  Why the Truth Might Get You Fired October 29, ...
3  Videos 15 Civilians Killed In Single US Airstr...
4  Print \nAn Iranian woman has been sentenced to...
5  In these trying times, Jackie Mason is the Voi...
6  Ever wonder how Britain’s most iconic pop pian...
7  PARIS  —   France chose an idealistic, traditi...
8  Donald J. Trump is scheduled to make a highly ...
9  A week before Michael T. Flynn resigned as nat...

In [23]:
train_df['text'][2188]

'\nThis article was written by John Vibes and originally published at Activist Post.com . \nEditor’s Comment: On the surface, there are good intentions here. The Bushes, like other patriotic establishment figures, always honor the troops, and count the sacrifices. Typically, they are stoic and praiseworthy of everything the military does. But underneath the surface, one has to wonder what is really going on inside the disturbed mind of George W. Bush – whose orders for an illegal war in Iraq, and a broader War on Terror in Afghanistan and elsewhere resulted in the deaths of thousands of U.S. troops, tens of thousands of wounded or injured and potentially more than a million civilians dead. \nDoes he feel remorse? Guilt? Has he admitted to his personal responsibility for what happened to their lives? Or do his paintings reveal a morbid sense of power over the lives of men and women whom he can only reflect upon and humanize after the fact? Considering that the Iraq war was based upon a 

**PRE PROCESSING OF DATA**

In [24]:
lemmatizer = WordNetLemmatizer()
stpwrds = list(stopwords.words('english'))

In [25]:
stpwrds

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

**LEMMATIZATION**

Lemmatization is the process of reducing words to their base or dictionary form, known as the "lemma." 
In natural language processing (NLP), lemmatization is an important technique used to normalize text data by removing 			  inflections, conjugations, and other variations of a word to bring them to their root form.

         example:the word "cats" can be lemmatized to "cat",
		 	    "running" to "run", 
		 	    "am" to "be"
		 	    "ate" to "eat"
Lemmatization is similar to stemming, but it produces more accurate results because it takes into account the context of the 			word in a sentence and uses more advanced linguistic analysis to identify the base form.
Lemmatization can improve the accuracy of text analysis and machine learning models that rely on text data.

In [ ]:
#lemmatization
for x in range(len(train_df)) :
    corpus = []
    review = train_df['text'][x]
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = review.lower()
    review = nltk.word_tokenize(review)
    for y in review :
        if y not in stpwrds :
            corpus.append(lemmatizer.lemmatize(y))
    review = ' '.join(corpus)
    train_df['text'][x] = review      

C:\Users\evilv\AppData\Local\Temp\ipykernel_4780\1796383705.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_df['text'][x] = review


In [ ]:
train_df['text'][2182]

**SPLITTING THE DATASET INTO TRAINING DATA & TEST DATA:**

In [ ]:
#splitting the dataset to training and test dataset,80% data to be training data &20% will be testing data.(ie test_size=0.2)
X_train, X_test, Y_train, Y_test = train_test_split(train_df['text'], label_train, test_size=0.2, random_state=0)

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
Y_train

**CONVERTING THE TEXTUAL DATA TO NUMERICAL DATA**

converting to textual data into numerical data/feature vector

1)**tf (term freequency)**:-which counts the number of time a perticular words is repearting in a documents.

2)**idf (inverse document freequency)**:-Measure the word is a rare word or common word in a document

In [ ]:
#converting to textual data into numerical data/feature vector

tfidf_v = TfidfVectorizer()
tfidf_X_train = tfidf_v.fit_transform(X_train)
tfidf_X_test = tfidf_v.transform(X_test)

In [ ]:
print(tfidf_X_train,tfidf_X_test)

In [ ]:
tfidf_X_train.shape

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

**CLASSIFICATION MODEL:-PASSIVE AGGRESSIVE CLASSIFIER**

Passive: if correct classification, keep the model; Aggressive: if incorrect classification, update to adjust to this misclassified example.

Passive-Aggressive algorithms are generally used for large-scale learning. It is one of the few 'online-learning algorithmns. In online machine learning algorithms, the input data comes in sequential order and the machine learning model is updated step-by-step, as opposed to batch learning, where the entire training dataset is used at once. This is very useful in situations where there is a huge amount of data and it is computationally infeasible to train the entire dataset because of the sheer size of the data. We can simply say that an online-leaming algorithm will get a training example, update the classifier, and then throw away the example.

In [ ]:
#model and model training

classifier = PassiveAggressiveClassifier()
classifier.fit(tfidf_X_train,Y_train)

**ACCURACY SCORE & CONFUSION MATRIX**

In [ ]:
#accuracy score on the test data
Y_pred = classifier.predict(tfidf_X_test)
score = metrics.accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {round(score*100,2)}%')
cm = metrics.confusion_matrix(Y_test, Y_pred)
plot_confusion_matrix(cm, classes=['FAKE Data', 'REAL Data'])

**CLASSIFICATION REPORT**

In [ ]:
print(classification_report(Y_test, Y_pred))

**SAVING & LOADING MODEL**

In [ ]:
#pickling the model into disk
pickle.dump(classifier,open('./model.pkl', 'wb'))
pickle.dump(tfidf_v,open('vector.pkl', 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('./model.pkl', 'rb'))
#vector_form = pickle.load(open('vector.pkl', 'rb'))

**BUILDING A PREDICTIVE SYSTEM**

In [ ]:
def fake_news_det(news):
    review = news
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = review.lower()
    review = nltk.word_tokenize(review)
    corpus = []
    for y in review :
        if y not in stpwrds :
            corpus.append(lemmatizer.lemmatize(y))     
    input_data = [' '.join(corpus)]
    vectorized_input_data = tfidf_v.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    if prediction[0] == 1:
        print("Prediction of the News :  Looking Fake⚠ News📰 ")
    else:
        print("Prediction of the News : Looking Real News📰 ")
      

In [ ]:
X_test

In [ ]:
X_test[16236]

In [ ]:
Y_test[16236]

In [ ]:
news=str(input("Enter the News:"))
fake_news_det(news)

**MANUAL PREDICTION**

In [ ]:
print(tfidf_X_test)

In [ ]:
print(Y_test)

In [ ]:
X_new=tfidf_X_test[3]
prediction = loaded_model.predict(X_new)

if (prediction[0] == 1):
	print("Prediction of the News :  Looking Fake⚠ News📰 ")
else:
	print("Prediction of the News : Looking Real News📰 ")

In [ ]:
print(Y_test[17543])